In [5]:
import numpy as np
import pandas as pd

In [6]:
# import data into dataframes

user_df = pd.read_csv('User.csv')
symptom_df = pd.read_csv('Symptom.csv')
period_df = pd.read_csv('Period.csv')

In [7]:
# clean up headers
period_df = period_df.rename(index=str, columns={"start_date": "period_start_cycle_start", "end_date": "period_end"})
user_df = user_df.rename(index=str, columns={"id":"user_id"})
period_df = period_df.rename(index=str, columns={"User_id":"user_id"})

In [8]:
# Merge onboarding data with period time data
df_new = pd.merge(user_df, period_df, on='user_id')

In [9]:
# Convert period_start to datetime and drop NaTs (since there are only 8)
df_new['period_start_cycle_start'] = pd.to_datetime(df_new['period_start_cycle_start'], errors='coerce', dayfirst=True)
df_new['period_end'] = pd.to_datetime(df_new['period_end'], errors='coerce', dayfirst=True)
df_new.dropna(subset=['period_start_cycle_start'], inplace=True)

In [10]:
# Convert onboarding data to timedeltas
df_new['cycle_length_initial'] = pd.to_timedelta(df_new['cycle_length_initial'], unit="D")
df_new['period_length_initial'] = pd.to_timedelta(df_new['period_length_initial'], unit="D")

In [11]:
# Input data to period end where there is a NaN
df_new.loc[pd.isnull(df_new.period_end), 'period_end'] = df_new.period_start_cycle_start + df_new.period_length_initial

In [12]:
# Calculate cycle and previous cycle data
df_new['prev_cycle_end_date'] = df_new.period_start_cycle_start - pd.Timedelta(days=1)
df_new['cycle_end_date'] = df_new.period_start_cycle_start + df_new.cycle_length_initial

In [13]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34934 entries, 0 to 34941
Data columns (total 9 columns):
user_id                     34934 non-null int64
dob                         4086 non-null object
cycle_length_initial        34934 non-null timedelta64[ns]
period_length_initial       34934 non-null timedelta64[ns]
id                          34934 non-null int64
period_start_cycle_start    34934 non-null datetime64[ns]
period_end                  34934 non-null datetime64[ns]
prev_cycle_end_date         34934 non-null datetime64[ns]
cycle_end_date              34934 non-null datetime64[ns]
dtypes: datetime64[ns](4), int64(2), object(1), timedelta64[ns](2)
memory usage: 3.9+ MB


In [14]:
df_new.to_csv("periods_cleaned.csv")

In [21]:
df_new.describe()

,user_id,cycle_length_initial,period_length_initial,id
count,34934.000000,34934.0,34934,34934.000000
mean,3427.903933,25.0,5 days 13:59:54.641323,19894.689185
std,1856.803985,0.0,2 days 04:55:12.081001,11015.079702
min,252.000000,25.0,2 days 00:00:00,352.000000
25%,1804.000000,25.0,4 days 00:00:00,10359.250000
50%,3364.000000,25.0,5 days 00:00:00,19964.500000
75%,4908.000000,25.0,7 days 00:00:00,29441.750000
max,7055.000000,25.0,51 days 00:00:00,38809.000000
